In [1]:
import os
from datetime import datetime
from time import sleep

import pandas as pd
from dotenv import load_dotenv
from py_github.py_github import PyGithub
import sqlite3
import requests


# load .env file
load_dotenv()
pygh_username = os.getenv("PYGH_USER")
pygh_token = os.getenv("PYGH_TOKEN")
pygh = PyGithub(pygh_username, pygh_token)

sql_conn = sqlite3.connect("./data/github_gql.db")


In [2]:
def build_query(cursor = None):

    cursor_string = f', after: "{cursor}"' if cursor else ''
    base_query =f"""query {{
      organization(login:"torqata") {{
            repositories(first:1{cursor_string}) {{
              pageInfo {{ endCursor }}
              edges {{
                node {{
                  id
              name
              pullRequests(first:100, states:[MERGED] orderBy:{{field: CREATED_AT, direction:DESC}}){{
                edges{{
                  node{{
                    id
                    number
                    title
                    changedFiles
                    createdAt
                    closedAt
                    merged
                    mergedAt
                    author {{
                    login
                    }}
                    timelineItems(last: 1, itemTypes: [PULL_REQUEST_REVIEW]) {{
                        nodes {{
                        ... on PullRequestReview {{
                            __typename
                            author {{
                              login
                                }}
                        createdAt
                        state
                            }}
                        }}
                      }}
                    comments(first:1) {{
                      totalCount
                      edges {{
                        node {{
                          createdAt
                        }}
                      }}
                    }}
                  }}
                }}
              }}
                }}
              }}
            }}
      }}
    }}"""

    return base_query



In [3]:

def get_all_repos(cursor=None, repos=[]):
    github_graphql = "https://api.github.com/graphql"
    base_query = build_query(cursor)
    response = requests.post(github_graphql, json={'query': base_query}, auth=(pygh_username, pygh_token))
    resp_dict = response.json()
    cursor = resp_dict['data']['organization']['repositories']['pageInfo']['endCursor']
    if len(resp_dict['data']['organization']['repositories']['edges']) > 0:
        repos.append(resp_dict['data']['organization']['repositories']['edges'][0])
    if cursor:
        repos = get_all_repos(cursor, repos)
    return repos

all_repos = get_all_repos()

# all_repos

In [4]:
len(all_repos)

145

In [18]:
import pytz

prs = []
for repo in all_repos:
    repo = repo['node']

    for pr in repo['pullRequests']['edges']:

        pr_dict = {'repo_name': repo['name']}
        pr = pr['node']
        if not pr['closedAt']:
            continue
        created_date = datetime.strptime(pr['createdAt'], '%Y-%m-%dT%H:%M:%SZ')
        created_day_of_week = created_date.isoweekday()

        pr_dict['number'] = pr['number']
        pr_dict['number_of_files_changes'] = pr['changedFiles']
        pr_dict['created_at'] = pr['createdAt']
        pr_dict['created_at_eastern'] = created_date.astimezone(pytz.timezone('US/Eastern'))
        pr_dict['created_day_week'] = created_day_of_week
        pr_dict['closed_at'] = pr['closedAt']
        pr_dict['comment_count'] = pr['comments']['totalCount']
        pr_dict['merged'] = pr['merged']
        pr_dict['merged_at'] = pr['mergedAt']

        if pr.get('author'):
            if pr['author']['login'] == 'dependabot':
                continue
            pr_dict['author'] = pr['author']['login']

        closed_time = None

        # get time to first approval on the pr. timeline items come back in asc date so we just need the first approved state
        pr_dict['first_approval_time'] = None
        pr_dict['first_approval_time_eastern'] = None
        pr_dict['seconds_to_first_approval'] = None

        if len(pr['timelineItems']['nodes']) > 0:
            timeline_items = pr['timelineItems']['nodes']
            for item in timeline_items:
                if item['state'] == 'APPROVED':
                    first_approval_time = datetime.strptime(item['createdAt'], '%Y-%m-%dT%H:%M:%SZ')
                    first_approval_time_eastern = first_approval_time.astimezone(pytz.timezone('US/Eastern'))
                    pr_dict['first_approval_time'] = first_approval_time
                    pr_dict['first_approval_time_eastern'] = first_approval_time_eastern
                    seconds_to_first_approval = first_approval_time - created_date
                    pr_dict['seconds_to_first_approval'] = seconds_to_first_approval.seconds


        # get the time till pr was merged
        pr_dict['seconds_to_merge'] = None
        if pr['mergedAt']:
            merged_date = datetime.strptime(pr['mergedAt'], '%Y-%m-%dT%H:%M:%SZ')
            time_to_merge = merged_date - created_date
            merged_time = time_to_merge.seconds
            pr_dict['seconds_to_merge'] = merged_time

        # time to first comment
        pr_dict['first_comment_time'] = None
        pr_dict['seconds_to_first_comment'] = None
        if pr['comments']['totalCount'] > 0:
            pr_dict['first_comment_time'] = pr['comments']['edges'][0]['node']['createdAt']
            first_comment_date = datetime.strptime(pr_dict['first_comment_time'], '%Y-%m-%dT%H:%M:%SZ')
            time_to_first_comment = first_comment_date - created_date
            pr_dict['seconds_to_first_comment'] = time_to_first_comment.seconds

        prs.append(pr_dict)

df = pd.DataFrame(prs)
df.to_sql("repo_prs", sql_conn, if_exists="replace")


2710